<a href="https://colab.research.google.com/github/trailblazerakash/MachineLearning/blob/master/AmericanExpress/amExModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
%matplotlib inline

from itertools import chain
from sklearn import preprocessing, ensemble
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [0]:
campaign= pd.read_csv('/content/drive/My Drive/amExprt/train/campaign_data.csv')
coupon_item=pd.read_csv('/content/drive/My Drive/amExprt/train/coupon_item_mapping.csv')
customer_demo=pd.read_csv('/content/drive/My Drive/amExprt/train/customer_demographics.csv')
customer_trans=pd.read_csv('/content/drive/My Drive/amExprt/train/customer_transaction_data.csv')
item_data=pd.read_csv('/content/drive/My Drive/amExprt/train/item_data.csv')
train=pd.read_csv('/content/drive/My Drive/amExprt/train/train.csv')


test=pd.read_csv('/content/drive/My Drive/amExprt/test.csv')


id             0
campaign_id    0
coupon_id      0
customer_id    0
dtype: int64